In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from datetime import datetime

from __future__ import print_function
import pickle

In [2]:
url = 'https://www.the-numbers.com/movie/budgets/all/701'

user_agent = {'User-agent': 'Mozilla/5.0'}

response  = requests.get(url, headers = user_agent)

page = response.text

soup = BeautifulSoup(page,"lxml")

In [3]:
table = soup.find('table')
table

<table>
<tr><th> </th><th>Release<br/>Date</th><th>Movie</th><th>Production<br/>Budget</th><th>Domestic<br/>Gross</th><th>Worldwide<br/>Gross</th></tr>
<tr><td class="data">701</td>
<td><a href="/box-office-chart/daily/1998/10/23">Oct 23, 1998</a></td>
<td><b><a href="/movie/Soldier#tab=summary">Soldier</a></b></td>
<td class="data"> $75,000,000</td>
<td class="data"> $14,623,082</td>
<td class="data"> $14,623,082</td>
</tr>
<tr><td class="data">702</td>
<td><a href="/box-office-chart/daily/1999/01/15">Jan 15, 1999</a></td>
<td><b><a href="/movie/Virus#tab=summary">Virus</a></b></td>
<td class="data"> $75,000,000</td>
<td class="data"> $14,010,690</td>
<td class="data"> $30,626,690</td>
</tr>
<tr><td class="data">703</td>
<td><a href="/box-office-chart/daily/2001/02/23">Feb 23, 2001</a></td>
<td><b><a href="/movie/Monkeybone#tab=summary">Monkeybone</a></b></td>
<td class="data"> $75,000,000</td>
<td class="data"> $5,409,517</td>
<td class="data"> $5,409,517</td>
</tr>
<tr><td class="da

In [4]:
rows = [row for row in table.find_all('tr')]

In [5]:
movies = {}

for row in rows[1:]:
    items = row.find_all('td')
    link = row.find('a')
    title, url = link.text, link['href']
    movies[title] = [url] + [i.text for i in items]
    
movies

{'Oct 23, 1998': ['/box-office-chart/daily/1998/10/23',
  '701',
  'Oct 23, 1998',
  'Soldier',
  '\xa0$75,000,000',
  '\xa0$14,623,082',
  '\xa0$14,623,082'],
 'Jan 15, 1999': ['/box-office-chart/daily/1999/01/15',
  '702',
  'Jan 15, 1999',
  'Virus',
  '\xa0$75,000,000',
  '\xa0$14,010,690',
  '\xa0$30,626,690'],
 'Feb 23, 2001': ['/box-office-chart/daily/2001/02/23',
  '703',
  'Feb 23, 2001',
  'Monkeybone',
  '\xa0$75,000,000',
  '\xa0$5,409,517',
  '\xa0$5,409,517'],
 'Aug 7, 2019': ['/box-office-chart/daily/2019/08/07',
  '704',
  'Aug 7, 2019',
  'PLAYMOBIL',
  '\xa0$75,000,000',
  '\xa0$1,115,008',
  '\xa0$16,286,699'],
 'Jan 13, 2022': ['/box-office-chart/daily/2022/01/13',
  '705',
  'Jan 13, 2022',
  'The 355',
  '\xa0$75,000,000',
  '\xa0$0',
  '\xa0$0'],
 'Jun 17, 2015': ['/box-office-chart/daily/2015/06/17',
  '706',
  'Jun 17, 2015',
  'Minions',
  '\xa0$74,000,000',
  '\xa0$336,045,770',
  '\xa0$1,159,631,140'],
 'Jan 28, 2015': ['/box-office-chart/daily/2015/01/28',


In [6]:
budget = pd.DataFrame(movies).T  #transpose
budget.columns = ['link_stub','rank_all_movies','release','title', 
                    'budget', 'domestic_gross','worldwide_gross']

budget.shape

(100, 7)

In [7]:
budget['budget'] = budget['budget'].str.replace('$', '')
budget['budget'] = budget['budget'].str.replace(',', '')
budget['budget'] = budget['budget'].astype(int)

In [8]:
budget.head()

,link_stub,rank_all_movies,release,title,budget,domestic_gross,worldwide_gross
"Oct 23, 1998",/box-office-chart/daily/1998/10/23,701,"Oct 23, 1998",Soldier,75000000,"$14,623,082","$14,623,082"
"Jan 15, 1999",/box-office-chart/daily/1999/01/15,702,"Jan 15, 1999",Virus,75000000,"$14,010,690","$30,626,690"
"Feb 23, 2001",/box-office-chart/daily/2001/02/23,703,"Feb 23, 2001",Monkeybone,75000000,"$5,409,517","$5,409,517"
"Aug 7, 2019",/box-office-chart/daily/2019/08/07,704,"Aug 7, 2019",PLAYMOBIL,75000000,"$1,115,008","$16,286,699"
"Jan 13, 2022",/box-office-chart/daily/2022/01/13,705,"Jan 13, 2022",The 355,75000000,$0,$0


In [9]:
budget.to_csv('budget8.csv')